In [29]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
hate_data = pd.read_csv("./hate_speech_binary_dataset.csv", delimiter=",") # 혐오 문장
genderbias_data = pd.read_csv('./genderbias.csv', sep=',')  # 여성 비하 문장
ilbe_data = pd.read_csv('./badword.csv',encoding='CP949') # 일베 문장

## 문장 전처리

In [31]:
# 혐오 문장 처리
hate_data.columns = ['comment', 'label'] # 컬럼 명 변경
hate_data = hate_data.astype({'comment': 'str'})
hate_data = hate_data[hate_data['label']==1]

In [32]:
# 여성 비하 문장 처리
del genderbias_data['bias']    #해당 데이터셋의 필요없는 열 제거
del genderbias_data['hate']    #해당 데이터셋의 필요없는 열 제거
genderbias_data['contain_gender_bias'] = genderbias_data['contain_gender_bias'].replace([False, True],[0,1])  # 구분하기 쉽게 기존의 표기를 0,1로 변경
# genderbias_data = genderbias_data[['contain_gender_bias', 'comments']]    #구분하기 쉽게 열의 순서를 변경

genderbias_data.columns = ['comment', 'label'] # 컬럼 명 변경
genderbias_data = genderbias_data[genderbias_data['label']==1]

In [33]:
# 일베 문장 처리
ilbe_data = ilbe_data[['v2', 'v1']]    #구분하기 쉽게 열의 순서를 변경
ilbe_data.columns = ['comment', 'label'] # 컬럼 명 변경
ilbe_data = ilbe_data[ilbe_data['label']==1]

In [34]:
# 데이터 개수
print("hate_data : %d" % len(hate_data))
print("genderbias_data : %d" % len(genderbias_data))
print("ilbe_data : %d" % len(ilbe_data))

hate_data : 100000
genderbias_data : 1232
ilbe_data : 2044


## 라벨링 및 합치기

In [35]:
# 렌덤하게 문장 추출 후 라벨링 거치는 함수
def random_labeling(data, classified_data, label_num, str_num):  # label_num : 라벨링 시킬 값, str_num 추출할 문장 개수
    random_data = classified_data.sample(n=str_num) # str_num 개의 행(문장) 랜덤 추출
    random_data.loc[random_data.label == 1, 'label'] = label_num # label_num으로 값 변경
    data = data.append(random_data)  # data 에 랜덤 추출된 데이터 추가
    return data

In [36]:
data_declaration = {
    'comment' : ["str_Declaration"],
    'label':[-1]
}


data = pd.DataFrame(data_declaration)
data

,comment,label
0,str_Declaration,-1


In [37]:
# 문장 개수를 모두 2000 개로 지정 각 
# 라벨은 혐오:0, 여성비하: 1, 일베: 2로 지정됨
data = random_labeling(data, hate_data, 0, 1200)
data = random_labeling(data, genderbias_data, 1, 1200)
data = random_labeling(data, ilbe_data, 2, 1200)

In [38]:
# 첫번째 행 제거
data = data.drop(0)

In [39]:
# 랜덤하게 섞기 
data = data.sample(frac=1).reset_index(drop=True)

In [40]:
data[:5]

,comment,label
0,한국 영화의 드문 수작,0
1,"김수미가 시어머니면 피곤할텐데. 요리도 받쳐줘야 이쁨받지 아들타령할듯,",1
2,상습 주작글 올리는 개새끼야,2
3,나는 장모년이 지랄하길래 장모님은 왜 그거에 예민하신데요? 이러니 아무말 못하더라,2
4,ㅇㅈㅋㅋ 속으로는 ㅈㄴ게 찔리는일게이들인데,2


## 중복 및 null 처리

In [41]:
def detect_overlap(data):
    exist_overlap = len(data)  # 데이터 전체 개수
    no_overlap = data['comment'].nunique()  # 중복 제거된 개수
    if exist_overlap != no_overlap:
        data.drop_duplicates(subset=['comment'], inplace=True)

In [42]:
detect_overlap(data)

In [43]:
# null 값 확인
data.isnull().values.any()

False

## 토큰화

In [44]:
def tokenize(x_data, tokenizer):
    tokenizer.fit_on_texts(x_data) # 데이터의 각 행별로 토큰화 수행
    return tokenizer.texts_to_sequences(x_data)    

In [45]:
x_data = data['comment']
y_data = data['label']

In [46]:
tokenizer = Tokenizer()
sequences = tokenize(x_data, tokenizer)

## 희귀단어 확인

In [47]:
def detect_rareword(tokenizer, threshold):
    word_to_index = tokenizer.word_index 
    total_cnt = len(word_to_index) # 단어의 수
    rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
    total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
    rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

    # 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        # 단어의 등장 빈도수가 threshold보다 작으면
        if(value < threshold):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value

    print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
    print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
    print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
    

In [48]:
# 희귀 단어 확인
detect_rareword(tokenizer, 2)

등장 빈도가 1번 이하인 희귀 단어의 수: 17539
단어 집합(vocabulary)에서 희귀 단어의 비율: 83.89457571988903
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 51.52619054613825


## 훈련 데이터, 테스트 데이터 분리

In [49]:
# 가장 긴 길이
def max_length(X_data):
    return max(len(l) for l in X_data)

# 훈련 학습 데이터 개수
def num_dataset(raio, X_data):
    num_train = int(len(X_data)*0.8)
    num_test = int(len(X_data) - num_train)
    return num_train, num_test


In [50]:
# 최대 길이 계산
X_data = sequences
maxlen = max_length(X_data)

In [51]:
# 라벨 데이터
Y_data = np.array(y_data)

In [52]:
Y_data

array([0, 1, 2, ..., 2, 0, 0])

In [53]:
# 훈련 및 학습 데이터 개수 비율 지정
numtrain, numtest = num_dataset(0.8, X_data)

In [54]:
# 가장 긴 길이로 맞추기
X_data = pad_sequences(X_data, maxlen = maxlen)

In [56]:
X_data

array([[    0,     0,     0, ...,  1058,  1641,   304],
       [    0,     0,     0, ...,  3372,  3373,  3374],
       [    0,     0,     0, ...,  3376,  3377,  3378],
       ...,
       [    0,     0,     0, ..., 20905,  2320,     3],
       [    0,     0,     0, ...,     0,     4, 20906],
       [    0,     0,     0, ...,  1017,  1601,   203]], dtype=int32)